# Hoofdstuk 15 - Databases

In de slides behorende bij hoofdstuk 15 wordt uitgebreid ingegaan op SQL en op database normalisatie. Als het goed is weten jullie dat allemaal nog van het vak Computer Forensics & Gegevensbeheer wat vorig jaar gegeven is.

Helaas staan in de slides geen Python voorbeelden hoe je SQL queries vanuit je eigen script kunt aanroepen.

Voor iedere database is er wel een Python module te vinden om hier mee te connecten. Deze zullen allemaal heel erg op elkaar lijken. In deze notebook zullen we gebruik maken van de bij jullie reeds bekende SQlite.

Allereerst moeten we de sqlite3 module importeren in ons script. Er is verder geen noodzaak om hiervoor iets te moeten installeren aangezien elke Python interpreter reeds wordt meegeleverd met deze module.

## Initialisatie

In [1]:
import sqlite3

Vervolgens moet een database bestand worden geopend. Vergelijk dit met het openen van een tekstbestand met de `open()` functie. We gebruiken hiervoor de `sqlite3.connect()` functie voor.

In [2]:
connection = sqlite3.connect('example.db')

In [3]:
type(connection)

sqlite3.Connection

## Bevragen van een database connectie

In [4]:
help(sqlite3.Connection.execute)

Help on method_descriptor:

execute(...)
    Executes a SQL statement. Non-standard.



Zoals je ziet is de `execute()` functie niet standaard voor een typische database connectie. Veruit de meeste database modules gebruiken een `cursor` object.

In [5]:
cursor = connection.cursor()

In [6]:
type(cursor)

sqlite3.Cursor

In [7]:
help(sqlite3.Cursor.execute)

Help on method_descriptor:

execute(...)
    Executes a SQL statement.



In [8]:
cursor.execute('SELECT * FROM Users')

Je verwacht op dit punt waarschijnlijk de inhoud van de tabel te zien, zoals je bij de sqlite command line tool gewend was.

De SQL query is weliswaar uitgevoerd, maar de uitkomst moet nog wel worden uitgelezen in Python, bijvoorbeeld door een `for` lusje.

In [9]:
for row in cursor:
    print(row)

('Charles', 'csev@umich.edu')
('Colleen', 'cvl@umich.edu')
('Sally', 'a1@umich.edu')
('Kristen', 'kf@umich.edu')
('Richard Brinkman', 'r.brinkman@saxion.nl')
('Brinkman', None)
('Brinkman', None)
('Brinkman', None)


Zoals je ziet gedraagt een `cursor` object zich (binnen een `for` lus) als een lijst van `tuple`'s.

Maar een `cursor` object kan veel meer:

In [10]:
help(sqlite3.Cursor)

Help on class Cursor in module sqlite3:

class Cursor(builtins.object)
 |  SQLite database cursor class.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  close(...)
 |      Closes the cursor.
 |  
 |  execute(...)
 |      Executes a SQL statement.
 |  
 |  executemany(...)
 |      Repeatedly executes a SQL statement.
 |  
 |  executescript(...)
 |      Executes a multiple SQL statements at once. Non-standard.
 |  
 |  fetchall(...)
 |      Fetches all rows from the resultset.
 |  
 |  fetchmany(...)
 |      Fetches several rows from the resultset.
 |  
 |  fetchone(...)
 |      Fetches one row from the resultset.
 |  
 |  setinputsizes(...

Feitelijk wordt bij gebruik van een `for` lus dus eerst de `fetchall()` functie aangeroepen:

In [11]:
cursor.execute('SELECT * FROM Users')
cursor.fetchall()

[('Charles', 'csev@umich.edu'),
 ('Colleen', 'cvl@umich.edu'),
 ('Sally', 'a1@umich.edu'),
 ('Kristen', 'kf@umich.edu'),
 ('Richard Brinkman', 'r.brinkman@saxion.nl'),
 ('Brinkman', None),
 ('Brinkman', None),
 ('Brinkman', None)]

Je kunt ook één voor één de rows opvragen:

In [12]:
cursor.execute('SELECT * FROM Users')
cursor.fetchone()

('Charles', 'csev@umich.edu')

In [13]:
cursor.fetchone()

('Colleen', 'cvl@umich.edu')

Het restant wat dan overblijft is dan:

In [14]:
cursor.fetchall()

[('Sally', 'a1@umich.edu'),
 ('Kristen', 'kf@umich.edu'),
 ('Richard Brinkman', 'r.brinkman@saxion.nl'),
 ('Brinkman', None),
 ('Brinkman', None),
 ('Brinkman', None)]

De elementen van een cursor zijn dus maar eenmalig op te vragen. Heb je ze dus vaker nodig in je Python script dan zul je de lijst in een variabele moeten opslaan.

Als een cursor alle elementen heeft ge`fetch`ed, dan blijft er een lege lijst over:

In [15]:
cursor.fetchall()

[]

In [16]:
print(cursor.fetchone())

None


## INSERT queries

Een enkele regel toevoegen aan een tabel gaat als volgt:

In [ ]:
cursor.execute('INSERT INTO Users(name, email) VALUES (?, ?)', ('Richard Brinkman', 'r.brinkman@saxion.nl'))

### SQL injection attack

Zoals je ziet gebruiken we placeholders i.p.v. direct een SQL query als

```sql
INSERT INTO Users(name, email) VALUES ('Richard Brinkman', 'r.brinkman@saxion.nl')
```

Als de data die je wilt inserten namelijk afkomstig is van een onbetrouwbare bron (bijvoorbeeld hacker die een formuliertje invoert op een website), wil je deze niet direct in de query zetten.

Beschouw deze gevaarlijke code:

In [ ]:
name = input('Wat is uw naam')
sql = "INSERT INTO Users(name) VALUES ('" + name + "');"
cursor.executescript(sql)

Zolang je "normale" data opgeeft gaat het weliswaar goed, maar wat als de gebruiker iets intypt als:

```sql
Brinkman'); DELETE FROM Users WHERE name = 'Ted' AND ('' == '
```

Je zou dan 2 queries uitvoeren namelijk:
```sql
INSERT INTO Users(name) VALUES ('Brinkman'); DELETE FROM Users WHERE name = 'Ted' AND ('' == '')
```

In [ ]:
name = input('Wat is uw naam')
sql = "INSERT INTO Users(name) VALUES ('" + name + "');"
print(sql)
cursor.executescript(sql)

Gebruik daarom *altijd* placeholders als een deel van je query komt van een variabele.

In [ ]:
name = input('Wat is uw naam')
cursor.execute('INSERT INTO Users(name) VALUES (?)', (name,))

In [ ]:
cursor.execute("SELECT * FROM Users")
cursor.fetchall()

## Meerdere regels tegelijk inserten

Stel we hebben al een lijst van gegevens. Deze kunnen we met één enkele SQL query invoeren:

In [ ]:
personen = [
    ('Brinkman', 'r.brinkman@saxion.nl'),
    ('van Tuinen', 'm.vantuinen@saxion.nl')
]
cursor.executemany('INSERT INTO Users(name, email) VALUES (?, ?)', personen)

In [ ]:
cursor.execute('SELECT * FROM Users')
cursor.fetchall()

## Commit

Alles wat je met een `connectie` object doet wordt niet direct weggeschreven naar schijf. Het bestand `example.db` is dus nog altijd niet veranderd dat gebeurt pas na het volgende commando: